In [11]:
import pandas as pd
pd.options.mode.chained_assignment = None
from ast import literal_eval

#General results RQ

flag_df = pd.read_csv('CSV/results-flags-MIO.csv') # This CSV may have the results from the heuristic using MIO
flag_df2 = pd.read_csv('CSV/results-flags-OTHERS.csv') # This CSV may have the results from the heuristic using the other algorithms
flag_df['ConfigurationId'] = flag_df['ConfigurationId'].apply(lambda x: x.split('-')[1])
flag_df2['ConfigurationId'] = flag_df2['ConfigurationId'].apply(lambda x: x.split('-')[1])
flag_df = flag_df[flag_df['ConfigurationId'] == 'MIO']
flag_df = pd.concat([flag_df, flag_df2], ignore_index=True)


flag_df['GBOSFlags'] = flag_df['GBOSFlags'].apply(literal_eval)
flag_df = flag_df[['ConfigurationId', 'TargetModule','GBOSFlags', 'Coverage', 'AlgorithmIterations']]

In [12]:
import statistics
x = flag_df['TargetModule'].value_counts()
print("Number of modules with more than 10 runs: ",len(x[x > 10]))
print("Average runs per module: ", statistics.mean(x.values))

Number of modules with more than 10 runs:  554
Average runs per module:  77


In [13]:
import shutil

configurationIDs = ['MIO', 'DYNAMOSA', 'MOSA', 'WHOLE_SUITE']

important_modules = set()

all_flags = set()

values = [] 

table_template = '''
\\begin{{tabular}}{{lcccc}}\\toprule
    \\multicolumn{{1}}{{c}}{{\\# Modules that}}    & DynaMOSA   & MOSA      & WHOLE SUITE   & MIO     \\\\ \\midrule
    had correct runs                               & \\({6}\\)  & \\({12}\\) & \\({18}\\)     & \\({0}\\) \\\\
    had logs from the heuristic                    & \\({7}\\)  & \\({13}\\) & \\({19}\\)     & \\({1}\\) \\\\
    created a template                             & \\({8}\\)  & \\({14}\\) & \\({20}\\)    & \\({2}\\) \\\\ 
    added a template                               & \\({9}\\)  & \\({15}\\) & \\({21}\\)    & \\({3}\\) \\\\ 
    had no coverage goals                          & \\({10}\\)  & \\({16}\\) & \\({22}\\)    & \\({4}\\) \\\\ 
    had runtime errors                             & \\({11}\\)  & \\({17}\\) & \\({23}\\)    & \\({5}\\) \\\\ 
    \\bottomrule
  \\end{{tabular}}
'''

for config in configurationIDs:
    config_flag_df = flag_df[flag_df['ConfigurationId'] == config]

    flags_CONFIG = config_flag_df.loc[lambda df: df['GBOSFlags'] != set()]
    flags_CONFIG['Created'] = flags_CONFIG['GBOSFlags'].apply(lambda x: 'Chromosome Template created' in x)
    flags_CONFIG['Added'] = flags_CONFIG['GBOSFlags'].apply(lambda x:'Chromosome Template added into population' in x)
    flags_CONFIG['ModuleNotFound'] = flags_CONFIG['GBOSFlags'].apply(lambda x:'Module not found' in x)
    flags_CONFIG['NoCoverageGoals'] = flags_CONFIG['GBOSFlags'].apply(lambda x:'No coverage goals' in x or 'max() arg is an empty sequence' in x)
    flags_CONFIG['PynguinBug'] = flags_CONFIG['GBOSFlags'].apply(lambda x:'maximum recursion depth exceeded' in x or 'Bug in Pynguin!' in x or 'The current thread shall not be executed any more, thus I kill it.' in x)

    amount_modules = len(set(config_flag_df['TargetModule']))
    with_flags = len(set(flags_CONFIG['TargetModule']))
    trivial_cov = len(set(flags_CONFIG.loc[lambda df: df['GBOSFlags'] == set()].loc[lambda df: df['Coverage'] == 1.0].loc[lambda df: df['AlgorithmIterations'] == 0]))
    created = len(set(flags_CONFIG[flags_CONFIG['Created'] == True]['TargetModule']))
    added = len(set(flags_CONFIG[flags_CONFIG['Added'] == True]['TargetModule']))
    mnf = len(set(flags_CONFIG[flags_CONFIG['ModuleNotFound'] == True]['TargetModule']))
    ncg = len(set(flags_CONFIG[flags_CONFIG['NoCoverageGoals'] == True]['TargetModule']))
    pb = len(set(flags_CONFIG[flags_CONFIG['PynguinBug'] == True]['TargetModule']))
    
    important_modules.update(set(flags_CONFIG[flags_CONFIG['Added'] == True]['TargetModule']))

    values += [amount_modules, with_flags, created, added, ncg, pb]

    print("-"*60)
    print("Total modules 588")
    print("# Modules with at least one correct run in {0}:".format(config).ljust(70, " "),"{0}".format(amount_modules))
    print("# Modules with no logs in {0}:".format(config).ljust(70, " "),"{0}".format(amount_modules - with_flags))
    print("# Modules with no logs & trivial coverage in {0}:".format(config).ljust(70, " "),"{0}".format(trivial_cov))
    print("# Modules with logs in {0}:".format(config).ljust(70, " "),"{0}".format(with_flags))
    print("# Modules that created a test template in {0}:".format(config).ljust(70, " "),"{0}".format(created))
    print("# Modules that added a test template to the pop. in {0}:".format(config).ljust(70, " "),"{0}".format(added))
    print("# Modules that did not have coverage goals in {0}:".format(config).ljust(70, " "),"{0}".format(ncg))
    print("# Modules that had a ModuleNotFound exception in {0}:".format(config).ljust(70, " "),"{0}".format(mnf))
    print("# Modules that had a Pynguin related bug in {0}:".format(config).ljust(70, " "),"{0}".format(pb))

table = open('Figures/flags-modules.tex', 'w')
table.write(table_template.format(*values))
table.close()
shutil.copyfile('Figures/flags-modules.tex', '../Document/Figures/Results/flags-modules.tex')

------------------------------------------------------------
Total modules 588
# Modules with at least one correct run in MIO:                        527
# Modules with no logs in MIO:                                         104
# Modules with no logs & trivial coverage in MIO:                      10
# Modules with logs in MIO:                                            423
# Modules that created a test template in MIO:                         355
# Modules that added a test template to the pop. in MIO:               353
# Modules that did not have coverage goals in MIO:                     52
# Modules that had a ModuleNotFound exception in MIO:                  10
# Modules that had a Pynguin related bug in MIO:                       42
------------------------------------------------------------
Total modules 588
# Modules with at least one correct run in DYNAMOSA:                   553
# Modules with no logs in DYNAMOSA:                                    154
# Modules with no log

'../Document/Figures/Results/flags-modules.tex'

In [14]:
thesis_df = pd.read_csv('CSV/results-thesis4.csv')
default_df = thesis_df[thesis_df['ConfigurationId'].str.match('PYNGUIN')]
default_df['ConfigurationId'] = default_df['ConfigurationId'].apply(lambda x: x.split('-')[1])
default_df['SearchTime'] = thesis_df['SearchTime'].apply(lambda x: int(x/10**9))
default_df['TotalTime'] = thesis_df['TotalTime'].apply(lambda x: int(x/10**9))
default_df['GBOSGenerationTime'] = thesis_df['GBOSGenerationTime'].apply(lambda x: int(x/10**9))
default_df['GBOSDecisionTime'] = thesis_df['GBOSDecisionTime'].apply(lambda x: int(x/10**9))

In [20]:
import json 

configurationIDs = ['MIO', 'DYNAMOSA', 'MOSA', 'WHOLE_SUITE']

important_modules = set()

better_cov = set()
less_it = set()

ratio_file = open('filtered-ratio.json', 'r')
ratio = dict(json.load(ratio_file))

for config in configurationIDs:
    filtered_gbos = flag_df[flag_df['ConfigurationId'] == config]
    filtered_default = default_df[default_df['ConfigurationId'] == config]

    filtered_gbos = filtered_gbos.loc[lambda df: df['GBOSFlags'] != set()]
    filtered_gbos['Added'] = filtered_gbos['GBOSFlags'].apply(lambda x:'Chromosome Template added into population' in x)

    module_set_default = set(filtered_default['TargetModule'])
    module_set_gbos = set(filtered_gbos[filtered_gbos['Added'] == True]['TargetModule'])#module_set_gbos = set(filtered_gbos['TargetModule'])
    intersection_set = module_set_default.intersection(module_set_gbos)

    grouped_filtered_default = filtered_default[filtered_default['TargetModule'].isin(intersection_set)][['Coverage', 'AlgorithmIterations', 'TargetModule']].groupby('TargetModule').mean()
    grouped_flags_CONFIG = filtered_gbos[filtered_gbos['TargetModule'].isin(intersection_set)][['Coverage', 'AlgorithmIterations', 'TargetModule']].groupby('TargetModule').mean()

    for module in intersection_set:
        gbos_cov = grouped_flags_CONFIG.loc[module]['Coverage']
        def_cov = grouped_filtered_default.loc[module]['Coverage']
        gbos_it = grouped_flags_CONFIG.loc[module]['AlgorithmIterations']
        def_it = grouped_filtered_default.loc[module]['AlgorithmIterations']

        runs_gbos = filtered_gbos[filtered_gbos['TargetModule'] == module].shape[0]
        runs_default = filtered_default[filtered_default['TargetModule'] == module].shape[0]

        if def_it == 0 or runs_gbos < 10:
            continue

        diff = ((gbos_it - def_it)/def_it)*100
        run_diff = abs(runs_gbos - runs_default)
        
        if gbos_cov > def_cov:
            better_cov.add((module, ratio[module], gbos_cov - def_cov, run_diff, config))
        if gbos_cov == def_cov and diff < 0:
            less_it.add((module, ratio[module], diff, run_diff, config))

better_cov_sorted = sorted(better_cov, key=lambda x: x[2])
better_cov_filtered = [elem for elem in better_cov if elem[1] > 0.0]
better_cov_filtered_sorted = sorted(better_cov_filtered, key=lambda x: x[2])
better_cov_sorted.reverse()
better_cov_filtered_sorted.reverse()

less_it_sorted = sorted(less_it, key=lambda x: x[2])
less_it_filtered = [elem for elem in less_it if elem[1] > 0.0]
less_it_filtered_sorted = sorted(less_it_filtered, key=lambda x: x[2])

print("Module Name".ljust(50," "), "OLR".center(5, " "), "Cov/Iter Diff".center(20, " "), "# Run diff".center(10, " "), "# Config".center(15, " "))
print(str(better_cov_sorted[0][0]).ljust(50, " "), "{0:.3f}".format(better_cov_sorted[0][1]).center(5, " "), "{0:.3f}".format(better_cov_sorted[0][2]).center(20, " "), str(better_cov_sorted[0][3]).center(10, " "), str(better_cov_sorted[0][4]).center(15, " "))
print(str(better_cov_filtered_sorted[0][0]).ljust(50, " "), "{0:.3f}".format(better_cov_filtered_sorted[0][1]).center(5, " "), "{0:.3f}".format(better_cov_filtered_sorted[0][2]).center(20, " "), str(better_cov_filtered_sorted[0][3]).center(10, " "), str(better_cov_filtered_sorted[0][4]).center(15, " "))
print(str(less_it_sorted[0][0]).ljust(50, " "), "{0:.3f}".format(less_it_sorted[0][1]).center(5, " "), "{0:.3f}".format(less_it_sorted[0][2]).center(20, " "), str(less_it_sorted[0][3]).center(10, " "), str(less_it_sorted[0][4]).center(15, " "))
print(str(less_it_filtered_sorted[0][0]).ljust(50, " "), "{0:.3f}".format(less_it_filtered_sorted[0][1]).center(5, " "), "{0:.3f}".format(less_it_filtered_sorted[0][2]).center(20, " "), str(less_it_filtered_sorted[0][3]).center(10, " "), str(less_it_filtered_sorted[0][4]).center(15, " "))

Module Name                                         OLR     Cov/Iter Diff     # Run diff     # Config   
setuptools.command.setopt                          0.000        0.066             9          DYNAMOSA   
pkg_resources._vendor.jaraco.functools             0.750        0.047             6            MOSA     
slugify.__main__                                   0.000       -99.587            10           MIO      
setuptools._vendor.importlib_resources._legacy     0.786       -86.457            16       WHOLE_SUITE  


In [ ]:
filtered_gbos = flag_df[flag_df['ConfigurationId'] == 'MIO']
filtered_gbos = filtered_gbos.loc[lambda df: df['GBOSFlags'] != set()]
filtered_gbos['Added'] = filtered_gbos['GBOSFlags'].apply(lambda x:'Chromosome Template added into population' in x)

filtered_default = default_df[default_df['ConfigurationId'] == 'MIO']

filtered_gbos[filtered_gbos['TargetModule'] == 'slugify.__main__'][['TargetModule', 'Coverage', 'AlgorithmIterations']].head()



,TargetModule,Coverage,AlgorithmIterations
9455,slugify.__main__,0.478261,2003
9456,slugify.__main__,0.478261,2002
9457,slugify.__main__,0.478261,1003
9458,slugify.__main__,0.478261,1003
9459,slugify.__main__,0.478261,2004


In [ ]:
filtered_default[filtered_default['TargetModule'] == 'slugify.__main__'][['TargetModule', 'Coverage', 'AlgorithmIterations']].head()

,TargetModule,Coverage,AlgorithmIterations
71719,slugify.__main__,0.478261,425368
71720,slugify.__main__,0.478261,398698
71721,slugify.__main__,0.478261,397547
71722,slugify.__main__,0.478261,413273
71723,slugify.__main__,0.478261,410558
